In [3]:
import datetime
import os
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

%load_ext blackcellmagic

In [4]:
data_path = Path(os.path.abspath('')) / 'data'
sentiment_path = data_path /  'sentiment' / 'ea.csv'
stock_path = data_path /  'ea.csv'

In [5]:
ea = pd.read_csv(sentiment_path).drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
ea = ea[ea["created_utc"] != "created_utc"]
ea["created_strf"] = (
    ea["created_utc"]
    .astype(int)
    .apply(lambda x: datetime.datetime.fromtimestamp(x).strftime("%Y-%d-%m"))
)

In [6]:
min_time = ea['created_strf'].min()
max_time = ea['created_strf'].max()

In [7]:
ea.groupby('created_strf')['body'].count()

created_strf
2019-01-01    927
2019-01-02     18
2019-01-03     68
2019-01-04      3
2019-01-05     21
             ... 
2021-31-03     45
2021-31-05    132
2021-31-07     35
2021-31-08     67
2021-31-10     24
Name: body, Length: 1051, dtype: int64

In [8]:
stock_data = pd.read_csv(stock_path)

In [9]:
stock_data

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,sma_5,...,wma_50,bb_50_up,bb_50_down,sma_100,ema_100,wma_100,bb_100_up,bb_100_down,macd,signal
0,0,2018-12-11,47.637469,47.921902,46.803791,47.274574,12474126,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.274574,NaN,NaN,NaN,0.000000,0.000000
1,1,2018-12-12,48.108254,49.255792,47.490349,47.519775,11453591,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.398401,NaN,NaN,NaN,0.019560,0.003912
2,2,2018-12-13,47.814015,48.314221,46.804772,47.578621,7755190,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.459680,NaN,NaN,NaN,0.039356,0.011001
3,3,2018-12-14,46.558591,47.519775,46.068191,46.833214,7161356,0.0,0,NaN,...,NaN,NaN,NaN,NaN,47.298334,NaN,NaN,NaN,-0.005045,0.007792
4,4,2018-12-17,46.686096,47.490351,45.744528,46.176083,10064952,0.0,0,47.076453,...,NaN,NaN,NaN,NaN,47.064817,NaN,NaN,NaN,-0.092195,-0.012206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,752,2021-12-06,57.529999,58.450001,57.240002,58.090000,9408963,0.0,0,57.724000,...,67.369043,86.515702,55.688299,76.6782,75.181358,72.416444,93.540384,59.816016,-4.139811,-3.948483
753,753,2021-12-07,58.919998,59.270000,58.282001,58.770000,11553517,0.0,0,57.758000,...,66.885435,86.532406,55.019194,76.3479,74.856381,72.061826,93.306999,59.388801,-3.954544,-3.949695
754,754,2021-12-08,58.779999,60.009998,58.029999,59.270000,9785536,0.0,0,58.156001,...,66.434228,86.437224,54.431976,76.0376,74.547740,71.723650,93.100452,58.974748,-3.724440,-3.904644
755,755,2021-12-09,58.595001,59.810001,58.520000,59.070000,8402308,0.0,0,58.512000,...,65.988557,86.259199,53.877601,75.7132,74.241250,71.387658,92.820961,58.605439,-3.517670,-3.827249


In [24]:
def prepare_data(X, X_cols, y_col="Close"):
    X_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    return X_scaler.fit_transform(X.loc[:, X_cols]), y_scaler.fit_transform(
        X.loc[:, y_col]
    )


def split_data(X, step_train, step_predict, y_col="Close"):
    data_len = X.shape[0]
    X_list = []
    Y_preds_real_list = []
    Y_whole_real_list = []
    for i in range(data_len):
        X_step = X.loc[i : i + step_train, :]
        Y_pred_real = X.loc[i + step_train : i + step_train + step_predict - 1, y_col]
        Y_whole_real = X.loc[i: i + step_train + step_predict - 1, y_col]
        X_list.append(X_step)
        Y_preds_real_list.append(Y_pred_real)
        Y_whole_real_list.append(Y_whole_real)
    return X_list, Y_preds_real_list, Y_whole_real_list

In [25]:
STEP_TRAIN = 30
STEP_PREDICT = 1

In [26]:
X_list, Y_preds_real_list, Y_whole_real_list= split_data(stock_data, STEP_TRAIN, STEP_PREDICT)

In [27]:
X_list[0].shape

(31, 36)

In [28]:
Y_preds_real_list[0].shape

(1,)

In [29]:
Y_whole_real_list[0].shape

(31,)